Setting up and importing:

In [ ]:
# Lightning related imports

# !pip install git+https://github.com/PyTorchLightning/pytorch-lightning fsspec --no-deps --target=$nb_path
# !pip install git+https://github.com/PyTorchLightning/pytorch-lightning
!pip install torchtext==0.8.0 torch==1.7.0 torchvision==0.8 pytorch-lightning==1.2.2
!pip install neptune-client==0.5.1

# !pip install git+https://github.com/PyTorchLightning/pytorch-lightning
# https://github.com/PyTorchLightning/pytorch-lightning/issues/6210

     |████████████████████████████████| 7.0MB 9.3MB/s 
     |█████████▉                      | 238.6MB 1.4MB/s eta 0:06:36

In [ ]:
# !pip install torch==1.7.0 torchvision==0.8 torchtext==0.8
!pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.7.0+cu101.html
!pip install -q torch-geometric
!pip install ogb

In [4]:
%load_ext autoreload
%autoreload 2
import pytorch_lightning
print(pytorch_lightning.__version__)
from pytorch_lightning import Trainer
from pytorch_lightning.loggers.neptune import NeptuneLogger
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

import json
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from scikitplot.metrics import plot_confusion_matrix
from google.colab import files
from lightning_model import LightningPAN


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
1.2.2


In [5]:
# Testing if the lightning_model script gets refreshed correctly
LightningPAN.training_epoch_end("hello",'Hle')


basemodel-posweight-meanloss


TypeError: ignored

In [7]:

# loading params
with open('parameters.json') as json_file:
    parameters = json.load(json_file)
print(parameters)


datasetname = parameters["dataset_name"]
phi = parameters["phi"]
runs = parameters["runs"]
batch_size = parameters["batch_size"]
filter_size = parameters["maximum_path_size"]+1
learning_rate = parameters["learning_rate"]
weight_decay = parameters["weight_decay"]
pool_ratio = parameters["pool_ratio"] # fraction of nodes to use for pooled feature array
pan_pool_weight = parameters['pan_pool_weight'] # importance weight on global matrix X
criterion_pos_weight = parameters['criterion_pos_weight'] # Over-weightage on positive labels 
nhid = parameters["nhid"]
epochs = parameters['epochs']; epochs=3



{'dataset_name': 'ogbg-molhiv', 'phi': 0.3, 'runs': 1, 'num_workers': 0, 'batch_size': 128, 'maximum_path_size': 3, 'learning_rate': 0.001, 'weight_decay': 0.0005, 'pool_ratio': 0.5, 'pan_pool_weight': 0.5, 'criterion_pos_weight': 30, 'nhid': 512, 'epochs': 150}


Running the model 5 times and saving model checkpoints by epochs:

In [8]:
num_iters = 5
exp_id_dict = {} 
for i in range(num_iters):
  print("***********************RUNNING MODEL FOR ITERATION " + str(i) + '*********************')
  # LearningRateMonitor_Params = {'logging_interval': 'epoch'}
  # lr_logger = LearningRateMonitor(**LearningRateMonitor_Params)

  # Create model checkpoint directory
  os.system("mkdir models_"+str(i))

  checkpoint_callback = ModelCheckpoint(
      monitor='validation_loss',
      save_top_k=-1, save_last=True, save_weights_only=True,
      mode='min',
      dirpath='/content/models_'+ str(i) + '/',
      filename='PAN-{epoch:02d}-{validation_loss:.4f}-{rocauc_eval:.4f}',
      period=1
  )

  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  print('Device: {}'.format(device))

  model = LightningPAN(9, 1, nhid=nhid, 
                      ratio=pool_ratio, pan_pool_weight=pan_pool_weight,
                      criterion_pos_weight=criterion_pos_weight,
                      filter_size=filter_size)
  neptune_logger = NeptuneLogger(
                  api_key="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiZDMyZTE4YmUtZTU1Yy00MDhhLTgyMzMtOTJiNzJjMTdkYWJkIn0=",
                  project_name='leyldy/drugs-PAN',
                  close_after_fit=False,
                  params=parameters, # your hyperparameters, immutable
                  tags=['base model', 'filter size across all layers', "criterion pos weight", "iter " + str(i)],  # tags
                  upload_source_files=["parameters.json", "lightning_model.py"]
                  )
  
  exp_id_dict[neptune_logger.experiment.id] = i

  trainer = Trainer(
      max_epochs=epochs,
      logger=neptune_logger,
      callbacks=[checkpoint_callback],
      gpus=1,
      # fast_dev_run=True,
  )

  model.cuda()
  trainer.fit(model)

  trainer.test(model)
  neptune_logger.experiment.stop()

  os.system("zip -r /content/models_checkpoint_"+ str(i) +'.zip '+'/content/models_'+str(i))
  files.download("/content/models_checkpoint_"+ str(i) + ".zip") 


NeptuneLogger will work in online mode


Device: cuda
https://ui.neptune.ai/leyldy/drugs-PAN/e/DRUG-31


GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name         | Type              | Params
---------------------------------------------------
0 | criterion    | BCEWithLogitsLoss | 0     
1 | atom_encoder | AtomEncoder       | 5.5 K 
2 | conv1        | PANConv           | 16.9 K
3 | pool1        | PANPooling        | 518   
4 | conv2        | PANConv           | 131 K 
5 | pool2        | PANPooling        | 262   
6 | conv3        | PANConv           | 32.9 K
7 | pool3        | PANPooling        | 134   
8 | lin1         | Linear            | 129   
---------------------------------------------------
187 K     Trainable params
0         Non-trainable params
187 K     Total params
0.751     Total estimated model params size (MB)
  0%|          | 0/3 [00:00<?, ?it/s]

Downloaded 0.00 GB: 100%|██████████| 3/3 [00:01<00:00,  2.90it/s]


Extracting dataset/hiv.zip
Processing...
Loading necessary files...
This might take a while.


...

basemodel-posweight-meanloss


Saving latest checkpoint...



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'rocauc_test': 0.6453175997991464, 'test_loss': 1.2965342565016313}
--------------------------------------------------------------------------------


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Save model checkpoints
for i in range(num_iters):
  os.system("zip -r /content/models_checkpoint_dup_"+ str(i) +'.zip '+'/content/models_'+str(i))
  files.download("/content/models_checkpoint_dup_"+ str(i) + ".zip") 

Seeing which model gave best validation results:

In [9]:
import neptune
myproj = neptune.init("leyldy/drugs-PAN", api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiZDMyZTE4YmUtZTU1Yy00MDhhLTgyMzMtOTJiNzJjMTdkYWJkIn0=')
full_results = pd.DataFrame()
for myexp in myproj.get_experiments()[-num_iters:]:
  seed_result = myexp.get_numeric_channels_values('validation_loss', 'rocauc_eval')
  seed_result['exp_id'] = myexp.id
  seed_result['iter'] = seed_result['exp_id'].map(exp_id_dict)
  full_results = pd.concat([full_results, seed_result], axis=0)

full_results.rename(columns={'x': 'epoch'}, inplace=True)
print(full_results.isnull().sum())
# full_results.to_csv('/content/full_validation_results.csv', index=False)
# files.download('/content/full_validation_results.csv')

print(full_results.shape)
full_results.head()

epoch              0
validation_loss    0
rocauc_eval        0
exp_id             0
iter               0
dtype: int64
(15, 5)


,epoch,validation_loss,rocauc_eval,exp_id,iter
0,0.0,1.495442,0.409851,DRUG-31,0
1,1.0,1.086888,0.520849,DRUG-31,0
2,2.0,1.114528,0.539295,DRUG-31,0
0,0.0,1.084618,0.512670,DRUG-32,1
1,1.0,1.104293,0.608689,DRUG-32,1


Method 1: Summarizing via choosing best iteration epoch per iteration:

In [10]:
idx = full_results.groupby(['exp_id'])['validation_loss'].transform(min) == full_results['validation_loss']

best_epoch_per_iter = full_results[idx]
best_epoch_per_iter

,epoch,validation_loss,rocauc_eval,exp_id,iter
1,1.0,1.086888,0.520849,DRUG-31,0
2,2.0,1.064640,0.668576,DRUG-32,1
1,1.0,1.044167,0.491068,DRUG-33,2
0,0.0,1.013119,0.660935,DRUG-34,3
1,1.0,1.092381,0.519480,DRUG-35,4


In [17]:
from os import listdir
from os.path import isfile, join

test_results = {'iter': [], 'epoch': [], 'rocauc_test': [], 'test_loss': []}
model_paths = ['/content/models_' + str(i) for i in range(num_iters)]
for i, mod_path in enumerate(model_paths):
  onlyfiles = [f for f in listdir(mod_path) if isfile(join(mod_path, f))]
  best_epoch = int(best_epoch_per_iter.loc[best_epoch_per_iter['iter']==i, 'epoch'])
  onlyfiles = [ckpt_nm for ckpt_nm in onlyfiles if 'epoch='+"{:02d}".format(best_epoch) in ckpt_nm]
  ckpt_nm = onlyfiles[0]
  print(i, ckpt_nm)

  ckpt_mod = LightningPAN.load_from_checkpoint(checkpoint_path=mod_path+"/"+ckpt_nm)
  ckpt_test_results = trainer.test(ckpt_mod)

  test_results['iter'] += [i]
  test_results['epoch'] += [best_epoch]
  test_results['rocauc_test'] += [ckpt_test_results[0]['rocauc_test']]
  test_results['test_loss'] += [ckpt_test_results[0]['test_loss']]

test_results_df = pd.DataFrame.from_dict(test_results).reset_index(drop=True)
id_to_exp_map = {val: key for key,val in exp_id_dict.items()}
test_results_df['exp_id'] = test_results_df['iter'].map(id_to_exp_map)

test_results_df.to_csv('/content/test_results.csv', index=False)
files.download('/content/test_results.csv')
test_results_df

0 PAN-epoch=01-validation_loss=1.09-rocauc_eval=0.52.ckpt



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'rocauc_test': 0.5149017941636571, 'test_loss': 1.316548932682384}
--------------------------------------------------------------------------------
1 PAN-epoch=02-validation_loss=1.06-rocauc_eval=0.67.ckpt



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'rocauc_test': 0.6036790977037023, 'test_loss': 1.291359007358551}
--------------------------------------------------------------------------------
2 PAN-epoch=01-validation_loss=1.04-rocauc_eval=0.49.ckpt



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'rocauc_test': 0.5906014021128256, 'test_loss': 1.281768829533548}
--------------------------------------------------------------------------------
3 PAN-epoch=00-validation_loss=1.01-rocauc_eval=0.66.ckpt



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'rocauc_test': 0.6613569207593812, 'test_loss': 1.258723952553489}
--------------------------------------------------------------------------------
4 PAN-epoch=01-validation_loss=1.09-rocauc_eval=0.52.ckpt



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'rocauc_test': 0.5296046659842794, 'test_loss': 1.3132888728922063}
--------------------------------------------------------------------------------


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,iter,epoch,rocauc_test,test_loss,exp_id
0,0,1,0.514902,1.316549,DRUG-31
1,1,2,0.603679,1.291359,DRUG-32
2,2,1,0.590601,1.281769,DRUG-33
3,3,0,0.661357,1.258724,DRUG-34
4,4,1,0.529605,1.313289,DRUG-35


Method 2: Summarizing per epoch across iterations

In [ ]:
# summ_results = full_results.groupby(['epoch']).mean().reset_index()
# summ_results.sort_values(['validation_loss'], ascending=True, inplace=True)
# summ_results.head()

,epoch,rocauc_eval,validation_loss
3,3.0,0.656087,34.415566
4,4.0,0.654051,34.751163
2,2.0,0.646194,35.530787
1,1.0,0.616151,38.533437
0,0.0,0.580398,41.788674


Finally, calculating test set results for this best validation performance epoch and downloading it:

In [ ]:
# from os import listdir
# from os.path import isfile, join

# best_epoch = int(summ_results['epoch'].values[0])

# test_results = {'rocauc_test': [], 'test_loss': []}
# model_paths = ['/content/models_' + str(i) for i in range(num_iters)]
# for mod_path in model_paths:
#   onlyfiles = [f for f in listdir(mod_path) if isfile(join(mod_path, f))]
#   onlyfiles = [ckpt_nm for ckpt_nm in onlyfiles if 'epoch='+"{:02d}".format(best_epoch) in ckpt_nm]
#   ckpt_nm = onlyfiles[0]
#   print(ckpt_nm)

#   ckpt_mod = LightningPAN.load_from_checkpoint(checkpoint_path=mod_path+"/"+ckpt_nm)
#   ckpt_test_results = trainer.test(ckpt_mod)

#   test_results['rocauc_test'] += [ckpt_test_results[0]['rocauc_test']]
#   test_results['test_loss'] += [ckpt_test_results[0]['test_loss']]

# test_results_df = pd.DataFrame.from_dict(test_results).reset_index()
# test_results_df.rename(columns={'index': 'iter'}, inplace=True)
# test_results_df.to_csv('/content/test_results.csv', index=False)
# files.download('/content/test_results.csv')
# test_results_df

PAN-epoch=03-validation_loss=33.18-rocauc_eval=0.69.ckpt



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'rocauc_test': 0.6376339828888159, 'test_loss': 42.10786324739456}
--------------------------------------------------------------------------------
PAN-epoch=03-validation_loss=34.33-rocauc_eval=0.66.ckpt



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'rocauc_test': 0.6550029934915699, 'test_loss': 40.97424465417862}
--------------------------------------------------------------------------------
PAN-epoch=03-validation_loss=35.74-rocauc_eval=0.62.ckpt



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'rocauc_test': 0.6086328434307344, 'test_loss': 42.3417871594429}
--------------------------------------------------------------------------------


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,iter,rocauc_test,test_loss
0,0,0.637634,42.107863
1,1,0.655003,40.974245
2,2,0.608633,42.341787


Miscellaneous

In [ ]:
# Command to keep colab running (run on browser console (i.e. chrome, cmd+shift+c))
# function ClickConnect(){
#     console.log("Clicked on connect button"); 
#     document.querySelector("colab-connect-button").click()
# }
# setInterval(ClickConnect,60000)

# var startClickConnect = function startClickConnect(){
#     var clickConnect = function clickConnect(){
#         console.log("Connnect Clicked - Start");
#         document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
#         console.log("Connnect Clicked - End"); 
#     };

#     var intervalId = setInterval(clickConnect, 60000);

#     var stopClickConnectHandler = function stopClickConnect() {
#         console.log("Connnect Clicked Stopped - Start");
#         clearInterval(intervalId);
#         console.log("Connnect Clicked Stopped - End");
#     };

#     return stopClickConnectHandler;
# };

# var stopClickConnect = startClickConnect();

# stopClickConnect();


Exploratory Data Analysis:

In [ ]:
# # Importing dataset to do some quick EDA
# from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
# # Load the dataset 
# dataset = PygGraphPropPredDataset(name='ogbg-molhiv')

# split_idx = dataset.get_idx_split()

# # Check task type
# print('Task type: {}'.format(dataset.task_type))

In [ ]:
# directed_g, undirected_g = 0, 0
# neg_graphs = []; pos_graphs = []
# num_edges_lst = [];
# num_nodes_lst = [];
# for i, curr_g in enumerate(dataset):
#   num_edges_lst.append(curr_g.num_edges)
#   num_nodes_lst.append(curr_g.num_nodes)
#   if curr_g.is_directed():
#     directed_g += 1
#   else:
#     undirected_g += 1
  
#   if curr_g.y.item() == 0:
#     neg_graphs.append(i)
#   elif curr_g.y.item() == 1:
#     pos_graphs.append(i)
#   else:
#     continue

In [ ]:
# print("Total number of graphs:", len(dataset))
# print("Number of classes:", dataset.num_classes)
# print("Dimension of node features:", dataset.num_node_features)
# print("Dimension of edge features:", dataset.num_edge_features)
# print("Number of positive labels:", dataset.data.y.sum().item())
# print("Number of negative labels:", dataset.data.y.shape[0] - dataset.data.y.sum().item())
# print("Min, Mean, Max # edges:", np.min(num_edges_lst), round(np.mean(num_edges_lst),2), np.max(num_edges_lst))
# print("Min, Mean, Max # nodes:", np.min(num_nodes_lst), round(np.mean(num_nodes_lst),2), np.max(num_nodes_lst))
# print("Number of directed graphs:", directed_g)
# print("Number of undirected graphs:", undirected_g)